In [1]:
import numpy as np
import pandas as pd

# Модель для прогнозирования, например ARIMA
from sktime.forecasting.arima import ARIMA
# Визуализация временных рядов
from sktime.utils.plotting import plot_series
# Модули для кросс-валидации
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError # Метрики MSE, MAE, MAPE
# Поиск оптимальных гиперпараметров по сетке
from sktime.forecasting.model_selection import ForecastingGridSearchCV

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

In [2]:
y = np.log(web.DataReader(name='GDP', data_source='fred', start='1995-01-01'))
y.index = y.index.to_period(freq='M') # замена индекса на периодический. Иначе кросс-валидация не будет работать
# длина ряда
len(y)

120

## Проведем Grid Search

In [5]:
# Зададим метод прогнозирования
forecaster = ARIMA()

# разбиваем параметры кросс-валидации
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=80, step_length=1)

# Задаём сетку для значений параметров модели в виде словаря
# будем менять параметры order и trend
param_grid = {'order':[(1,0,1), (1,1,0), (1,1,1), (1,2,0), (0,1,1), (0,2,0), (1,1,1)],
               'trend': ['ct', 'c', 'c', 'n', 'c', 'n', 'c']}

# инициализируем метрики
metric = MeanSquaredError(square_root=False)

# Grid search - метод поиска
gscv = ForecastingGridSearchCV(forecaster=forecaster, param_grid=param_grid, cv=cv_strategy, scoring=metric)

gscv.fit(y)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=array([1, 2, 3, 4, 5]),
                                                   initial_window=80),
                        forecaster=ARIMA(),
                        param_grid={'order': [(1, 0, 1), (1, 1, 0), (1, 1, 1),
                                              (1, 2, 0), (0, 1, 1), (0, 2, 0),
                                              (1, 1, 1)],
                                    'trend': ['ct', 'c', 'c', 'n', 'c', 'n',
                                              'c']},
                        scoring=MeanSquaredError())

In [6]:
# Параметры оптимальной модели
gscv.get_fitted_params()['best_forecaster']

ARIMA(order=(0, 1, 1), trend='c')

In [8]:
# Зададим метод прогнозирования
forecaster = ARIMA()

# разбиваем параметры кросс-валидации
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=80, step_length=1)

# Задаём сетку для значений параметров модели в виде словаря
# будем менять параметры order и trend
param_grid = {'order':[(1,0,1), (1,1,0), (1,1,1), (1,2,0), (0,1,1), (0,2,0), (1,1,1)],
               'trend': ['ct', 'c', 'c', 'n', 'c', 'n', 'c']}

# инициализируем метрики
metric = MeanAbsoluteError()

# Grid search - метод поиска
gscv = ForecastingGridSearchCV(forecaster=forecaster, param_grid=param_grid, cv=cv_strategy, scoring=metric)

gscv.fit(y)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=array([1, 2, 3, 4, 5]),
                                                   initial_window=80),
                        forecaster=ARIMA(),
                        param_grid={'order': [(1, 0, 1), (1, 1, 0), (1, 1, 1),
                                              (1, 2, 0), (0, 1, 1), (0, 2, 0),
                                              (1, 1, 1)],
                                    'trend': ['ct', 'c', 'c', 'n', 'c', 'n',
                                              'c']},
                        scoring=MeanAbsoluteError())

In [9]:
# Параметры оптимальной модели
gscv.get_fitted_params()['best_forecaster']

ARIMA(order=(0, 1, 1), trend='c')

### Построим оптимальную модель ARIMA для USM2

In [15]:
y = np.log(web.DataReader(name='M2SL', data_source='fred', start='1995-01-01'))
y.index = y.index.to_period(freq='M') # замена индекса на периодический. Иначе кросс-валидация не будет работать
# длина ряда
len(y)

361

In [16]:
# Зададим метод прогнозирования
forecaster = ARIMA()

# разбиваем параметры кросс-валидации
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=100, step_length=5)

# Задаём сетку для значений параметров модели в виде словаря
# будем менять параметры order и trend
param_grid = {'order':[(2,0,2), (2,1,0), (2,1,1), (1,2,0)],
               'trend': ['ct', 'c', 'n', 'n']}

# инициализируем метрики
metric = MeanAbsoluteError()

# Grid search - метод поиска
gscv = ForecastingGridSearchCV(forecaster=forecaster, param_grid=param_grid, cv=cv_strategy, scoring=metric)

gscv.fit(y)

ForecastingGridSearchCV(cv=ExpandingWindowSplitter(fh=array([1, 2, 3, 4, 5]),
                                                   initial_window=100),
                        forecaster=ARIMA(),
                        param_grid={'order': [(2, 0, 2), (2, 1, 0), (2, 1, 1),
                                              (1, 2, 0)],
                                    'trend': ['ct', 'c', 'n', 'n']},
                        scoring=MeanAbsoluteError())

In [17]:
gscv.get_fitted_params()['best_forecaster']

ARIMA(order=(2, 1, 0), trend='c')

In [18]:
gscv.get_fitted_params()['best_forecaster'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  361
Model:               SARIMAX(2, 1, 0)   Log Likelihood                1439.438
Date:                Tue, 25 Mar 2025   AIC                          -2870.876
Time:                        10:57:20   BIC                          -2855.331
Sample:                    01-31-1995   HQIC                         -2864.695
                         - 01-31-2025                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0021      0.000      5.924      0.000       0.001       0.003
ar.L1          0.6450      0.022     29.857      0.000       0.603       0.687
ar.L2         -0.0556      0.028     -1.966      0.049      -0.111      -0.000
sigma2      1.964e-05   6.21e-07     31.643      0.000    1.84e-05    2.09e-05
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):              7039.91
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               2.22   Skew:                             2.41
Prob(H) (two-sided):                  0.00   Kurtosis:                        24.12
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""